In [1]:
from colorrefinement import *
from c2 import *
import json, csv
import networkx as nx
import numpy as np
from GC2NodeClassifier import GC2NodeClassifier

In [2]:
#GuardedExistsEq(10, Var.y, E(Var.x, Var.y)),
#GuardedExistsGeq(4, Var.y, Or(GuardedExistsLeq(7, Var.x, E(Var.x, Var.y)), GuardedExistsGeq(13, Var.x, E(Var.x, Var.y)))),
#GuardedExistsGeq(9, Var.y, GuardedExistsLeq(13, Var.x, GuardedExistsGeq(7, Var.y, E(Var.x, Var.y)))),

In [3]:
formula = GuardedExistsGeq(4, Var.y, Or(GuardedExistsLeq(7, Var.x, E(Var.x, Var.y)), GuardedExistsGeq(13, Var.x, E(Var.x, Var.y))))
graph = nx.fast_gnp_random_graph(1000, 0.01)
for i in range(graph.number_of_nodes()):
    result = formula.evaluate(graph, i)
    graph.nodes[i][repr(formula)] = result

In [4]:
""" n = 0
for i in range(graph.number_of_nodes()):
    graph.nodes[i]["red"] = np.random.randint(0, 2)
    graph.nodes[i]["blue"] = np.random.randint(0, 2)
for i in range(n):
    for j in range(graph.number_of_nodes()):
        graph.nodes[j][f"P{i}"] = np.random.randint(0, 2)
for i in range(graph.number_of_nodes()):
    graph.nodes[i]["red_xor_blue"] = graph.nodes[i]["red"] ^ graph.nodes[i]["blue"] """

' n = 0\nfor i in range(graph.number_of_nodes()):\n    graph.nodes[i]["red"] = np.random.randint(0, 2)\n    graph.nodes[i]["blue"] = np.random.randint(0, 2)\nfor i in range(n):\n    for j in range(graph.number_of_nodes()):\n        graph.nodes[j][f"P{i}"] = np.random.randint(0, 2)\nfor i in range(graph.number_of_nodes()):\n    graph.nodes[i]["red_xor_blue"] = graph.nodes[i]["red"] ^ graph.nodes[i]["blue"] '

In [5]:
""" print(f"{"red_xor_blue"}:")
print(f"{len([node for node in graph.nodes if graph.nodes[node]["red_xor_blue"]])/graph.number_of_nodes()}") """

' print(f"{"red_xor_blue"}:")\nprint(f"{len([node for node in graph.nodes if graph.nodes[node]["red_xor_blue"]])/graph.number_of_nodes()}") '

In [6]:
from cProfile import Profile
from pstats import SortKey, Stats
with Profile() as prof:
    clf = GC2NodeClassifier(graph, [], repr(formula), evaluation_depth=3)
    (
        Stats(prof)
        .strip_dirs()
        .sort_stats(SortKey.CUMULATIVE)
        .print_stats(40)
    )

         24440 function calls (24410 primitive calls) in 0.022 seconds

   Ordered by: cumulative time
   List reduced from 269 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5125    0.003    0.000    0.008    0.000 convert_matrix.py:590(<genexpr>)
     5125    0.004    0.000    0.005    0.000 reportviews.py:845(__iter__)
        1    0.000    0.000    0.003    0.003 threading.py:616(wait)
        1    0.000    0.000    0.003    0.003 GC2NodeClassifier.py:12(__init__)
      5/1    0.000    0.000    0.003    0.003 GC2NodeClassifier.py:64(update)
        1    0.000    0.000    0.002    0.002 _coo.py:132(__init__)
        1    0.000    0.000    0.002    0.002 _coo.py:273(_check)
       26    0.002    0.000    0.002    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.000    0.000    0.002    0.001 {method 'max' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.002    0.001 _methods.py:39(_amax)
        7

In [7]:
clf.accuracy()

0.696

In [8]:
clf.formula()

⊤

In [34]:
clf.extend(0, only_misses=True)
clf.formula()

∃≥4x.(E(x, y) ∧ ((∃≤11x.E(x, y) ∨ ∃≥13x.E(x, y)) ∧ (∃≥12x.E(x, y) ∨ ∃≤7x.E(x, y))))

In [33]:
clf.extend(1, only_misses=False)
clf.features_n[1]

[(∃≥12x.E(x, y) ∧ ¬∃≥11x.E(x, y)),
 (¬∃≥17x.E(x, y) ∧ ∃≥16x.E(x, y)),
 (¬∃≥14x.E(x, y) ∧ ∃≥13x.E(x, y)),
 (¬(¬∃≥16x.E(x, y) ∧ ∃≥14x.E(x, y)) ∧ (¬∃≥17x.E(x, y) ∧ ∃≥13x.E(x, y))),
 (¬∃≥16x.E(x, y) ∧ ∃≥15x.E(x, y)),
 (¬∃≥17x.E(x, y) ∧ ∃≥15x.E(x, y)),
 (¬(¬∃≥15x.E(x, y) ∧ ∃≥14x.E(x, y)) ∧ (¬∃≥16x.E(x, y) ∧ ∃≥13x.E(x, y))),
 (¬(¬∃≥15x.E(x, y) ∧ ∃≥14x.E(x, y)) ∧ (¬∃≥17x.E(x, y) ∧ ∃≥13x.E(x, y))),
 (¬∃≥13x.E(x, y) ∧ ∃≥12x.E(x, y)),
 (¬(¬∃≥16x.E(x, y) ∧ ∃≥13x.E(x, y)) ∧ (¬∃≥17x.E(x, y) ∧ ∃≥12x.E(x, y))),
 (¬∃≥14x.E(x, y) ∧ ∃≥12x.E(x, y)),
 (¬(¬∃≥16x.E(x, y) ∧ ∃≥14x.E(x, y)) ∧ (¬∃≥17x.E(x, y) ∧ ∃≥12x.E(x, y))),
 (¬(¬∃≥15x.E(x, y) ∧ ∃≥13x.E(x, y)) ∧ (¬∃≥16x.E(x, y) ∧ ∃≥12x.E(x, y))),
 (¬(¬∃≥15x.E(x, y) ∧ ∃≥13x.E(x, y)) ∧ (¬∃≥17x.E(x, y) ∧ ∃≥12x.E(x, y))),
 (¬(¬∃≥15x.E(x, y) ∧ ∃≥14x.E(x, y)) ∧ (¬∃≥16x.E(x, y) ∧ ∃≥12x.E(x, y))),
 (¬(¬∃≥15x.E(x, y) ∧ ∃≥14x.E(x, y)) ∧ (¬∃≥17x.E(x, y) ∧ ∃≥12x.E(x, y))),
 ((¬∃≥12x.E(x, y) ∧ ¬∃≥10x.E(x, y)) ∧ ∃≥9x.E(x, y)),
 (¬(¬∃≥16x.E(x, y) ∧ ¬(¬∃≥12x.E(x, y) ∧ ¬∃≥10

In [32]:
clf.combine(1)

In [12]:
len(clf.features_n[1])

[∃≥12x.E(x, y), ⊤]

In [13]:
clf.formula()

∃≥9x.E(x, y)

In [35]:
clf.accuracy()

1.0